In [ ]:
pip install neo4j rdflib

In [ ]:
from zipfile import ZipFile
import requests
from io import BytesIO
from neo4j import GraphDatabase, basic_auth
import json
from rdflib import Graph

###Functions to interact with Neo4j

In [ ]:
driver = GraphDatabase.driver(
  "bolt://<your_neo4j_ip>:7687",
  auth=basic_auth("your_usr", "your_pwd"))

def store_onto(driver, dbname, onto_as_string):
  onto_import_query = '''
  call n10s.rdf.import.inline($payload,'Turtle')
  '''

  with driver.session(database=dbname) as session:
    results = session.execute_write(
      lambda tx: tx.run(onto_import_query, payload=onto_as_string).data())
    for record in results:
      print(record)

def store_onto_delta(driver, dbname, deleted, added):
  onto_add_query = '''
  call n10s.rdf.import.inline($payload,'Turtle')
  '''
  onto_delete_query = '''
  call n10s.rdf.delete.inline($payload,'Turtle')
  '''
  with driver.session(database=dbname) as session:
    results = session.execute_write(
      lambda tx: tx.run(onto_add_query, payload=added).data())
    for record in results:
      print(record)
    results = session.execute_write(
      lambda tx: tx.run(onto_delete_query, payload=deleted).data())
    for record in results:
      print(record)

###Get the current version of the ontology from the dev tool and store it in Neo4j

In [ ]:
web_protege_url = "https://webprotege.stanford.edu/download?project={project_id}&format=ttl".format(project_id = input("WebProtege project ID: "))
web_protege_cookies = {'JSESSIONID': input("Session ID from cookies: ")}

In [ ]:
with ZipFile(BytesIO(requests.get(web_protege_url, cookies = web_protege_cookies).content)) as zp:
  for filename in zp.namelist():
    print(filename)
    store_onto(driver, "db_name", zp.read(filename).decode("utf-8"))

###Get the current ontology in Neo4j and load it into an RdfLib Graph object

In [ ]:
neo4j_rdf_endpoint = 'http://<your_neo4j_ip>:7474/rdf/<your_dbname>/cypher'
query_get_ontology = 'match (r:Resource) return r union match (:Resource)-[r]-() return r'

payload = { 'cypher' : query_get_ontology , 'format' : 'Turtle' }

response = requests.post(neo4j_rdf_endpoint, auth=('your_usr', 'your_pwd'), data = json.dumps(payload))
response.raise_for_status()  # raise an error on unsuccessful status codes

current = Graph()
current.parse(data=response.text, format='turtle')


In [ ]:
print(current.serialize())

### Get the current ontology in WebProtege and load it into an RdfLib Graph object

In [ ]:
new = Graph()
with ZipFile(BytesIO(requests.get(web_protege_url, cookies = web_protege_cookies).content)) as zp:
  for filename in zp.namelist():
    print(filename)
    new.parse(data=zp.read(filename).decode("utf-8"), format='turtle')

### Do delta analysis and persist as needed

In [ ]:
added = new - current
deleted = current - new
print("ADDED TRIPLES")
print(added.serialize(format="turtle"))
print("DELETED TRIPLES")
print(deleted.serialize(format="turtle"))

In [ ]:
store_onto_delta(driver, "db_name", deleted.serialize(format="turtle"), added.serialize(format="turtle"))